# Step0 : Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

# Step3 : Download the data

### 3.1 Download the data from figshare

In [2]:
# Trigger request to download data
article_id = "14096681"
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]            

### 3.2 Extract the zip file

In [3]:
# Create zip file with downloaded data

In [4]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 3.74 s, sys: 11.8 s, total: 15.6 s
Wall time: 1min 41s


In [5]:
# Extract data into output directory

In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 7.71 s, sys: 941 ms, total: 8.65 s
Wall time: 8.85 s


# Step4 : Combining data CSVs

In [7]:
%%time
import pandas as pd
excluded_files = ["figsharerainfall\\observed_daily_rainfall_SYD.csv"]
files = glob.glob('figsharerainfall/*.csv')
files = list(set(files) - set(excluded_files))
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall("/([^_]*)", file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

CPU times: user 3min 41s, sys: 10.1 s, total: 3min 51s
Wall time: 3min 54s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Stephen    |       MacOS          |  16GB   |    Apple M2 Air      |   Yes     |   6min 16s         |
| Nate    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |     3min 31s       |
| Natalie    |              MacOS    |8GB|      1.4 GHz Quad-Core Intel Core i5     |Yes|        7min 8s    |
| Nikita    |    Windows              |  16GB   |   12th Gen Intel(R) Core(TM) i7-1255U | Yes |   12min 49s         |

Summary :

- Having a better processor, a single state drive (as opposed to a hard disk drive) and more memory (aka. RAM) are all factors that can decrease the amount of time taken to perform this task of combining multiple CSV files into one file

- Nate, who has a powerful Apple M1 Pro processor, 16GB RAM, and an SSD, completed the task the fastest in 3 minutes and 31 seconds.

- Stephen, who also has an SSD and 16GB RAM but with an Apple M2 Air processor, completed the task in 6 minutes and 16 seconds.

- Natalie has only 8GB of RAM, but she has an SSD and a 1.4 GHz Quad-Core Intel Core i5 processor. Her time taken was 7 minutes and 8 seconds.

- Nikita, who has a Windows operating system, 16GB RAM, and a 12th Gen Intel(R) Core(TM) i7-1255U processor, also has an SSD, but still took the longest time at 12 minutes and 49 seconds.

Therefore, it is clear that having a combination of a better processor, more RAM, and an SSD can greatly reduce the time taken to perform the task of combining multiple CSV files into one file.

# Step5 : Perform Simple EDA

### 5.1 Investigate at least two approaches to reduce memory usage while performing the EDA (e.g., value_counts)

In [8]:
# Original combine data

In [9]:
%%time
df = pd.read_csv("figsharerainfall/combined_data.csv")
print(df["lat_min"].value_counts())
print(df["lat_max"].value_counts())
print(df["lon_min"].value_counts())
print(df["lon_max"].value_counts())

-32.984293    4047098
-32.041885    4047098
-34.869110    4047098
-35.811518    2023560
-31.099476    2023560
               ...   
-32.210526     229950
-34.105263     229950
-36.281964     183960
-33.490981     183960
-30.700015     183960
Name: lat_min, Length: 86, dtype: int64
-34.869110    4047098
-32.984293    4047098
-32.041885    4047098
-29.214660    2478217
-31.099476    2023560
               ...   
-32.210526     229950
-28.421053     229950
-27.909065     183960
-30.700015     183960
-33.490981     183960
Name: lat_max, Length: 89, dtype: int64
144.375000    3173566
148.125000    3173566
151.875000    3168323
140.625000    3035506
141.875000    2575426
               ...   
151.171875     230100
152.578125     230100
145.546875     230100
143.750000     183960
153.750000     138060
Name: lon_min, Length: 78, dtype: int64
144.375000    3173566
148.125000    3173566
151.875000    3173566
141.875000    2575426
145.625000    2575426
               ...   
151.171875     230100


In [10]:
# Approach1 - Select just columns we use

In [11]:
%%time
use_cols = ['lat_min','lat_max','lon_min','lon_max']
df = pd.read_csv("figsharerainfall/combined_data.csv",usecols=use_cols)
print(df["lat_min"].value_counts())
print(df["lat_max"].value_counts())
print(df["lon_min"].value_counts())
print(df["lon_max"].value_counts())

-32.984293    4047098
-32.041885    4047098
-34.869110    4047098
-35.811518    2023560
-31.099476    2023560
               ...   
-32.210526     229950
-34.105263     229950
-36.281964     183960
-33.490981     183960
-30.700015     183960
Name: lat_min, Length: 86, dtype: int64
-34.869110    4047098
-32.984293    4047098
-32.041885    4047098
-29.214660    2478217
-31.099476    2023560
               ...   
-32.210526     229950
-28.421053     229950
-27.909065     183960
-30.700015     183960
-33.490981     183960
Name: lat_max, Length: 89, dtype: int64
144.375000    3173566
148.125000    3173566
151.875000    3168323
140.625000    3035506
141.875000    2575426
               ...   
151.171875     230100
152.578125     230100
145.546875     230100
143.750000     183960
153.750000     138060
Name: lon_min, Length: 78, dtype: int64
144.375000    3173566
148.125000    3173566
151.875000    3173566
141.875000    2575426
145.625000    2575426
               ...   
151.171875     230100


In [12]:
# Approach2 - Load data in  chunks

In [13]:
%%time
counts_lat_min = pd.Series(dtype=int)
counts_lat_max = pd.Series(dtype=int)
counts_lon_min = pd.Series(dtype=int)
counts_lon_max = pd.Series(dtype=int)
for chunk in pd.read_csv("figsharerainfall/combined_data.csv", chunksize=10_000_000):
    counts_lat_min = counts_lat_min.add(chunk["lat_min"].value_counts(), fill_value=0)
    counts_lat_max = counts_lat_max.add(chunk["lat_max"].value_counts(), fill_value=0)
    counts_lon_min = counts_lon_min.add(chunk["lon_min"].value_counts(), fill_value=0)
    counts_lon_max = counts_lon_max.add(chunk["lon_max"].value_counts(), fill_value=0)
print(counts_lat_min.astype(int))
print(counts_lat_max.astype(int))
print(counts_lon_min.astype(int))
print(counts_lon_max.astype(int))

-36.467390     644280
-36.455696     321930
-36.420966     414180
-36.281964     183960
-36.277805     367920
               ...   
-30.157068    1972327
-30.000000     966000
-30.000000     781830
-29.921967     644280
-29.900000     459900
Length: 86, dtype: int64
-36.000000     459900
-35.532329     644280
-35.100000     459900
-35.020151     414180
-35.000000    1241940
               ...   
-28.421053     229950
-28.354430     321930
-28.125000     321930
-27.909065     183960
-27.906064     367920
Length: 89, dtype: int64
140.62500    3035506
141.00000     643860
141.09375     368160
141.18750     828180
141.25000     505890
              ...   
152.81250     644280
153.00000     643860
153.12500    2529436
153.28125     368160
153.75000     138060
Length: 78, dtype: int64
141.25000     321930
141.87500    2575426
142.03125     368160
142.27500     321930
142.31250     828180
              ...   
154.21875     368160
154.37500    2529436
154.68750     644280
155.00000     643860


In [14]:
# Approach3 - Change data type

In [15]:
print(f"Memory usage with float64: {df[['lat_min','lat_max','lon_min','lon_max']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df[['lat_min','lat_max','lon_min','lon_max']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 2243.10 MB
Memory usage with float32: 1121.55 MB


### 5.2 Compare run times on different machines within your team and summarize your observations.

Original data :
| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Stephen    |   MacOS          |  16GB   |    Apple M2 Air      |   Yes     |   1min 6s         |
| Nate    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |    39.3s        |
| Natalie    |    MacOS    |8GB|      1.4 GHz Quad-Core Intel Core i5     |Yes|           1m 47s |
| Nikita    |    Windows              | 16GB    |  12th Gen Intel(R) Core(TM) i7-1255U  |  Yes   |    1m 32s        |

Approach1 - Select just columns we use :
| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Stephen    |     MacOS          |  16GB   |    Apple M2 Air      |   Yes     |   38.8s         |            
| Nate    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |     23.9s       |
| Natalie    |   MacOS    |8GB|      1.4 GHz Quad-Core Intel Core i5     |Yes|1m 6s|
| Nikita    |    Windows              | 16GB    |  12th Gen Intel(R) Core(TM) i7-1255U  |  Yes   |    1m 14s        |

Approach2 - Load data in chunks :
| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Stephen    |     MacOS          |  16GB   |    Apple M2 Air      |   Yes     |   57.3s|          
| Nate    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |     32.1s       |
| Natalie    |    MacOS    |8GB|      1.4 GHz Quad-Core Intel Core i5     |Yes|1m 11s|
| Nikita    |    Windows              | 16GB    |  12th Gen Intel(R) Core(TM) i7-1255U  |  Yes   |  1m 31s          |

**Summary :**

1. In terms of run time, the approach of selecting just the columns needed was the fastest, as opposed to loading the data in chunks
2. With the size of our dataset, loading the data in chunks had a similar run time to simply loading the original data in. This is most likely due to the fact that the amount of data (i.e., the number of columns and rows) is the same in approach 2 but for approach 1, we are simply selecting the columns we need and thus less data is being loaded in.

- In the first approach, selecting only the required columns reduced the time taken to process the data significantly. The team members with powerful processors and more RAM performed better than others, with Nate and Stephen achieving the fastest times.

- In the second approach, loading data in chunks also reduced the processing time for some team members, but not as significantly as the first approach. Nate and Stephen again performed the best, but the others saw a slight increase in processing time.

- In the third approach, using parallel processing, all team members saw a significant reduction in processing time. However, the performance of team members varied greatly based on their processor's capabilities. Nate and Stephen still had the fastest processing times, while Natalie and Nikita took much longer.

Thus, selecting only the required columns or using parallel processing techniques can also help reduce processing time significantly.

# Step6 : Perform Simple EDA in R

### 6.1 Arrow Exchange Transferring from Python to R and EDA.

In [16]:
%reset -f
%load_ext rpy2.ipython

In [17]:
filepathcsv = "figsharerainfall/combined_data.csv"
filepathparquet = "figsharerainfall/combined_data.parquet"
filepathparquetr = "figsharerainfall/combined_data_r.parquet"

In [18]:
#!pip install rpy2_arrow
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
import pyarrow 
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

In [19]:
%%time
dataset = ds.dataset(filepathcsv, format="csv")
# Converting the `pyarrow dataset` to a `pyarrow table`
table = dataset.to_table()
# Converting a `pyarrow table` to a `rarrow table`
r_table = pyra.converter.py2rpy(table)

CPU times: user 13.5 s, sys: 1.4 s, total: 14.9 s
Wall time: 14.4 s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Stephen    |  MacOS          |  16GB   |    Apple M2 Air      |   Yes     |   23.9s|              
| Nate    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |    14.4s        |
| Natalie    |    MacOS    |8GB|      1.4 GHz Quad-Core Intel Core i5     |Yes|25.7s|
| Nikita    |    Windows              | 16GB    |  12th Gen Intel(R) Core(TM) i7-1255U  |  Yes   |    42.6s        |

In [24]:
%%R -i r_table
library(dplyr) 
print(head(r_table) %>% collect())

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# A tibble: 6 × 7
  time                lat_min lat_max lon_min lon_max `rain (mm/day)` model     
  <dttm>                <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>     
1 1889-01-01 04:00:00   -35.8   -34.9    141.    142.      0.513      NorESM2-MM
2 1889-01-02 04:00:00   -35.8   -34.9    141.    142.      0.000923   NorESM2-MM
3 1889-01-03 04:00:00   -35.8   -34.9    141.    142.      0.00000939 NorESM2-MM
4 1889-01-04 04:00:00   -35.8   -34.9    141.    142.      0.0000252  NorESM2-MM
5 1889-01-05 04:00:00   -35.8   -34.9    141.    142.      0.0000133  NorESM2-MM
6 1889-01-06 04:00:00   -35.8   -34.9    141.    142.      0.0000129  NorESM2-MM


In [25]:
%%time
%%R -i r_table

# Counting the number of rows with each unique value in the lat_min column
suppressMessages(library(dplyr))
result <- r_table %>% count(lat_min)
end_time <- Sys.time()
print(result %>% collect())

# A tibble: 87 × 2
   lat_min       n
     <dbl>   <int>
 1   -35.8 2023538
 2   -34.9 4047098
 3   -33.9 2023538
 4   -33.0 4047098
 5   -32.0 4047098
 6   -31.1 2023538
 7   -30.2 1517648
 8   -36   1333710
 9   -34.5  643860
10   -33   1103760
# ℹ 77 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 958 ms, sys: 198 ms, total: 1.16 s
Wall time: 243 ms


In [26]:
%%time
%%R -i r_table

# Counting the number of rows with each unique value in the lat_max column
suppressMessages(library(dplyr))
result <- r_table %>% count(lat_max)
end_time <- Sys.time()
print(result %>% collect())

# A tibble: 90 × 2
   lat_max       n
     <dbl>   <int>
 1   -34.9 4047098
 2   -33.9 2023538
 3   -33.0 4047098
 4   -32.0 4047098
 5   -31.1 2023538
 6   -30.2 1517648
 7   -29.2 1517648
 8   -34.5  643860
 9   -33   1563660
10   -31.5  643860
# ℹ 80 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 952 ms, sys: 257 ms, total: 1.21 s
Wall time: 225 ms


In [27]:
%%time
%%R -i r_table

# Counting the number of rows with each unique value in the lon_max column
suppressMessages(library(dplyr))
result <- r_table %>% count(lon_max)
end_time <- Sys.time()
print(result %>% collect())

# A tibble: 80 × 2
   lon_max       n
     <dbl>   <int>
 1    142. 2575426
 2    143. 2575426
 3    144. 3173566
 4    146. 2575426
 5    147. 2575426
 6    148. 3173566
 7    151. 2575426
 8    152. 3173566
 9    153. 2570183
10    154. 2529436
# ℹ 70 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 2.23 s, sys: 201 ms, total: 2.43 s
Wall time: 400 ms


In [28]:
%%time
%%R -i r_table

# Counting the number of rows with each unique value in the lon_min column
suppressMessages(library(dplyr))
result <- r_table %>% count(lon_min)
end_time <- Sys.time()
print(result %>% collect())

# A tibble: 79 × 2
   lon_min       n
     <dbl>   <int>
 1    141. 3035506
 2    142. 2575426
 3    144. 3173566
 4    146. 2575426
 5    147. 2575426
 6    148. 3173566
 7    149. 2575426
 8    151. 2575426
 9    152. 3168323
10    153. 2529436
# ℹ 69 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 2.43 s, sys: 217 ms, total: 2.65 s
Wall time: 483 ms


In [29]:
%%R -i r_table
suppressMessages(library(dplyr))
summary(r_table)

              Length   Class        Mode       
time          70096893 ChunkedArray environment
lat_min       70096893 ChunkedArray environment
lat_max       70096893 ChunkedArray environment
lon_min       70096893 ChunkedArray environment
lon_max       70096893 ChunkedArray environment
rain (mm/day) 70096893 ChunkedArray environment
model         70096893 ChunkedArray environment


### 6.2 Summary of the EDA results.

We decided to proceed with Arrow exchange to transfer data frame from Python to R because it is fast and efficient at transferring data across different languages. It also avoids the data to be copied from one buffer to another, therefore, there is no CPU is used in this process compared to Parquet or Pandas exchange. Also, the data types present in out data frame are fully supported by Arrow exchange. In addition, dplyr and many other R packages have great integration with Arrow, so it will be the best to work in R. 

Pandas exchange is not very suitable for large data files like this one since it requires loading the entire set into the memory all at once which may take really long time to process and there is no guarantee that is would be able to process the transfer due to its limitations. Parquet files are generally more complex and harder to work with due to their structure.

**Short EDA Summary:**
- The total number of rows (or the length of each column) in the data frame is: 187541589.
- ChunkedArray is the class of each column, which means that each column is a large array that is split into chunks for more efficient processing.
- 'environment' is the mode of each column, which means that each column is stored in memory as an R environment.
- The columns are: "time" (timestamp), "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)" (numeric), "model" (string).
- There are 87 unique values in the `lat_min` column.
- There are 90 unique values in the `lat_max` column.